In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

os.chdir('/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Scripts/Preprocessing/')
%run constants.py

In [39]:
def remove_na_df(df):
    processed_columns = []
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #drop na values in each column
    for column in df.columns:
        processed_column = df[column].dropna().reset_index(drop=True)
        processed_columns.append(processed_column)

    result_df = pd.concat(processed_columns, axis=1)
    result_df = result_df.dropna()
    result_df.columns = [f"{col}_cc" if col not in ['Epoch', 'Animal_ID'] else col for col in df.columns]
    return result_df

In [43]:
SYNGAP_1_ls = ['S7088']

In [45]:
results_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Connectivity/'
all_animals = []
for animal in SYNGAP_1_ls:
    print(animal)
    df = pd.read_csv(results_path + f'{animal}_cross_corr.csv')
    result_df = remove_na_df(df)
    all_animals.append(result_df)

S7088


In [42]:
all_animals_concat = pd.concat(all_animals)

In [43]:
all_animals_concat

,Epoch,Animal_ID,0_2_delta_cc,0_3_delta_cc,0_4_delta_cc,0_5_delta_cc,0_6_delta_cc,0_7_delta_cc,0_8_delta_cc,0_9_delta_cc,...,10_11_gamma_cc,10_12_gamma_cc,10_13_gamma_cc,10_15_gamma_cc,11_12_gamma_cc,11_13_gamma_cc,11_15_gamma_cc,12_13_gamma_cc,12_15_gamma_cc,13_15_gamma_cc
0,0,S7076,0.461879,0.664243,0.609397,0.722496,0.678915,0.756599,0.590976,0.545495,...,0.959293,0.956347,0.832111,0.963411,0.962864,0.909954,0.904592,0.917952,0.916966,0.789241
1,1,S7076,0.492624,0.803205,0.780439,0.848030,0.550736,0.733486,0.783964,0.653109,...,0.962405,0.967171,0.860727,0.959929,0.967912,0.913068,0.900649,0.927166,0.924946,0.800971
2,2,S7076,0.352703,0.633640,0.613290,0.775780,0.520753,0.726525,0.695055,0.484713,...,0.950481,0.961616,0.844206,0.950220,0.965284,0.908448,0.886479,0.920126,0.910530,0.786919
3,3,S7076,0.289614,0.538730,0.518668,0.570677,0.338995,0.625546,0.568553,0.410421,...,0.962381,0.971030,0.886777,0.967397,0.970528,0.933097,0.920285,0.936397,0.945508,0.865915
4,4,S7076,0.283445,0.580086,0.547229,0.782649,0.237778,0.714630,0.721157,0.607662,...,0.949363,0.965636,0.863627,0.959726,0.961616,0.928086,0.905895,0.917416,0.922833,0.824476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17275,17275,S7075,0.620075,0.844938,0.873833,0.947031,0.887903,0.931668,0.839925,0.926926,...,0.943929,0.961663,0.855583,0.932385,0.945417,0.911497,0.834677,0.898515,0.888990,0.726240
17276,17276,S7075,0.567652,0.589475,0.794845,0.879411,0.731347,0.783719,0.439369,0.736598,...,0.962005,0.964499,0.880682,0.957664,0.967597,0.930092,0.898830,0.936370,0.905310,0.788128
17277,17277,S7075,0.841146,0.921238,0.933721,0.958788,0.926685,0.951068,0.890482,0.942000,...,0.956026,0.963022,0.848064,0.947395,0.960067,0.907936,0.869897,0.908961,0.904300,0.733921
17278,17278,S7075,0.999200,0.999201,0.999201,0.999201,0.999201,0.999201,0.999201,0.999201,...,0.999200,0.999201,0.999199,0.999200,0.999200,0.999200,0.999198,0.999200,0.999199,0.999197


In [10]:
def analyse_plv(channel_labels, array, freq_band, num_channels = 14):
    all_epochs = []
    for epoch in array: 
        data_dict = {}
    
        # iterate through each epoch, ignoring self-pairs
        for idx_i, i in enumerate(channel_labels):
            for idx_j, j in enumerate(channel_labels):
                if i != j:
                    index = idx_i * num_channels + idx_j
                    pair_label = f"{i}_{j}_{freq_band}_plv"
                    data_dict[pair_label] = epoch[index]

        # convert the dictionary to df
        df = pd.DataFrame(data_dict)
        all_epochs.append(df)
    df_concat = pd.concat(all_epochs)
    return df_concat

def check_duplicate_columns(df_list):
    all_columns = []
    for df in df_list:
        all_columns.extend(df.columns)
    duplicates = [col for col in set(all_columns) if all_columns.count(col) > 1]
    print(duplicates)
    if duplicates:
        return duplicates
    

In [ ]:
results_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Connectivity/PLV/'
save_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Connectivity/PLV/animal/'
freq_bands = ['delta', 'theta', 'sigma', 'beta', 'gamma']
channel_labels = [0,2,3,4,5,6,7,8,9,10,11,12,13,15]
all_animals = []
for animal in analysis_ls:
    if animal in SYNGAP_1_ls:
        print(f'{animal} 1 ID')
        num_epochs = 17280
        all_bands = []
        for band in freq_bands:
            print(band)
            plv_np = np.load(f'{results_path}{animal}_{band}.npy')
            plv_band = analyse_plv(channel_labels = channel_labels, array = plv_np,
                               freq_band = band, num_channels = 14)
            all_bands.append(plv_band)
        animal_plv = pd.concat(all_bands, axis = 1)
        animal_plv.to_csv(f'{save_path}{animal}_plv.csv')
        
    elif animal in SYNGAP_2_ls:
        print(f'{animal} 2 ID')
        num_epochs = 34560
        all_bands = []
        for band in freq_bands:
            print(band)
            plv_np = np.load(f'{results_path}{animal}_{band}.npy')
            plv_band = analyse_plv(channel_labels = channel_labels, array = plv_np,
                               freq_band = band, num_channels = 14)
            all_bands.append(plv_band)
        animal_plv = pd.concat(all_bands[1:], axis = 1)
        animal_plv.to_csv(f'{save_path}{animal}_plv.csv')
    

S7088 1 ID
delta
theta
sigma
beta
gamma
S7092 1 ID
delta
theta
sigma
beta
gamma
S7086 2 ID
delta
theta
sigma
beta
gamma
S7063 2 ID
delta
theta
sigma
beta
gamma
S7064 2 ID
delta
theta
sigma
beta
gamma
S7069 2 ID
delta
theta
sigma
beta
gamma


In [15]:
all_bands[0]

,Animal_ID,Epochs
0,S7088,0
1,S7088,1
2,S7088,2
3,S7088,3
4,S7088,4
...,...,...
17275,S7088,17275
17276,S7088,17276
17277,S7088,17277
17278,S7088,17278


In [17]:
duplicate = check_duplicate_columns(all_bands)
duplicate

[]


In [24]:
all_bands_concat = pd.concat(all_bands[1:], axis = 1)

In [25]:
all_bands_concat

,0_2_delta_plv,0_3_delta_plv,0_4_delta_plv,0_5_delta_plv,0_6_delta_plv,0_7_delta_plv,0_8_delta_plv,0_9_delta_plv,0_10_delta_plv,0_11_delta_plv,...,15_4_gamma_plv,15_5_gamma_plv,15_6_gamma_plv,15_7_gamma_plv,15_8_gamma_plv,15_9_gamma_plv,15_10_gamma_plv,15_11_gamma_plv,15_12_gamma_plv,15_13_gamma_plv
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.594191,0.630741,0.588289,0.666420,0.724461,0.852994,0.610261,0.726709,0.812664,0.648553
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.574615,0.482699,0.442454,0.615069,0.648448,0.800886,0.549605,0.725397,0.754601,0.609599
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.525404,0.442098,0.395374,0.570745,0.634599,0.795102,0.433650,0.713371,0.756248,0.589896
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.605086,0.540185,0.462324,0.660871,0.684047,0.814315,0.568078,0.756125,0.795766,0.665350
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.618401,0.562490,0.495593,0.645870,0.665970,0.819192,0.566728,0.733562,0.800718,0.661162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.892827,0.905503,0.895128,0.920251,0.897673,0.957836,0.821785,0.929089,0.940922,0.895456
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.849284,0.866964,0.833208,0.876637,0.855658,0.933090,0.752804,0.887866,0.918220,0.862239
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.999999,1.000000,0.999998,0.999999,0.999999,0.999998,1.000000,0.999995,0.999999,0.999992


In [54]:
results_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Connectivity/PLV/'
save_path = '/home/melissa/PROJECT_DIRECTORIES/EEGFeatureExtraction/Results/Connectivity/PLV/animal/'
freq_bands = ['delta', 'theta', 'sigma', 'beta', 'gamma']


def load_and_process_plv(animal, num_epochs, freq_bands, results_path, channel_labels):
    all_bands = []
    # Create initial DataFrame with Animal_ID and Epochs
    initial_df = pd.DataFrame({
        'Animal_ID': [animal] * num_epochs,
        'Epochs': np.arange(num_epochs)
    })
    all_bands.append(initial_df)
    
    # Process each frequency band
    for band in freq_bands:
        print(f'Processing {band} band for animal {animal}')
        plv_np = np.load(f'{results_path}{animal}_{band}.npy')
        plv_band = analyse_plv(channel_labels=channel_labels, array=plv_np, freq_band=band, num_channels=14)
        all_bands.append(plv_band)
    
    # Concatenate all bands DataFrames horizontally
    animal_plv = pd.concat(all_bands, axis=1)
    
    return animal_plv

for animal in analysis_ls:
    if animal in SYNGAP_1_ls:
        print(f'{animal} 1 ID')
        num_epochs = 17280
        animal_plv = load_and_process_plv(animal, num_epochs, freq_bands, results_path, channel_labels)
        animal_plv.to_csv(f'{save_path}{animal}_plv.csv', index=False)
        
    elif animal in SYNGAP_2_ls:
        print(f'{animal} 2 ID')
        num_epochs = 34560
        animal_plv = load_and_process_plv(animal, num_epochs, freq_bands, results_path, channel_labels)
        animal_plv.to_csv(f'{save_path}{animal}_plv.csv', index=False)

S7088 1 ID
Processing delta band for animal S7088
Processing theta band for animal S7088
Processing sigma band for animal S7088
Processing beta band for animal S7088
Processing gamma band for animal S7088


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [44]:
df_id

,Animal_ID,Epochs
0,S7088,0
1,S7088,1
2,S7088,2
3,S7088,3
4,S7088,4
...,...,...
17275,S7088,17275
17276,S7088,17276
17277,S7088,17277
17278,S7088,17278


In [45]:
all_bands_concat

,0_2_delta_plv,0_3_delta_plv,0_4_delta_plv,0_5_delta_plv,0_6_delta_plv,0_7_delta_plv,0_8_delta_plv,0_9_delta_plv,0_10_delta_plv,0_11_delta_plv,...,15_4_gamma_plv,15_5_gamma_plv,15_6_gamma_plv,15_7_gamma_plv,15_8_gamma_plv,15_9_gamma_plv,15_10_gamma_plv,15_11_gamma_plv,15_12_gamma_plv,15_13_gamma_plv
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.594191,0.630741,0.588289,0.666420,0.724461,0.852994,0.610261,0.726709,0.812664,0.648553
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.574615,0.482699,0.442454,0.615069,0.648448,0.800886,0.549605,0.725397,0.754601,0.609599
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.525404,0.442098,0.395374,0.570745,0.634599,0.795102,0.433650,0.713371,0.756248,0.589896
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.605086,0.540185,0.462324,0.660871,0.684047,0.814315,0.568078,0.756125,0.795766,0.665350
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.618401,0.562490,0.495593,0.645870,0.665970,0.819192,0.566728,0.733562,0.800718,0.661162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.892827,0.905503,0.895128,0.920251,0.897673,0.957836,0.821785,0.929089,0.940922,0.895456
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.849284,0.866964,0.833208,0.876637,0.855658,0.933090,0.752804,0.887866,0.918220,0.862239
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.999999,1.000000,0.999998,0.999999,0.999999,0.999998,1.000000,0.999995,0.999999,0.999992


In [46]:
pd.concat([df_id, all_bands_concat], axis = 1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [9]:
for epoch in plv_np:
    print(epoch.flatten())
    break

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.40806497 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.7650011  0.6380963
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.79120272 0.43317244 0.86196868 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.84207927 0.41331744 0.78519335 0.80182561
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.56408445 0.2939714
 0.52915212 0.6049534  0.65364256 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.83039667 0.4585874  0.86691494 0.9121473  0.86261536 0.60026278
 0.         0.         0.         0.         0.         0.
 0.         0.         0.6

In [23]:

# Number of channels
num_channels = 14

# Channel labels
channel_labels = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15]

# Initialize a dictionary to store the pairs and their connectivity values
data_dict = {}

# Iterate through the array to extract pairs, ignoring self-pairs
for idx_i, i in enumerate(channel_labels):
    for idx_j, j in enumerate(channel_labels):
        if i != j:
            index = idx_i * num_channels + idx_j
            pair_label = f"{i}_{j}_plv"
            data_dict[pair_label] = epoch[index]

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data_dict)

# Display the DataFrame
print(df)

   0_2_plv  0_3_plv  0_4_plv  0_5_plv  0_6_plv  0_7_plv  0_8_plv  0_9_plv  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   0_10_plv  0_11_plv  ...  15_4_plv  15_5_plv  15_6_plv  15_7_plv  15_8_plv  \
0       0.0       0.0  ...  0.733284  0.835422  0.544905  0.767031  0.873877   

   15_9_plv  15_10_plv  15_11_plv  15_12_plv  15_13_plv  
0  0.909956   0.926128   0.814018   0.812233   0.685078  

[1 rows x 182 columns]


In [4]:
def analyse_plv(channel_labels, array, freq_band, num_channels = 14):
    all_epochs = []
    for epoch in array: 
        data_dict = {}
    
        # iterate through each epoch, ignoring self-pairs
        for idx_i, i in enumerate(channel_labels):
            for idx_j, j in enumerate(channel_labels):
                if i != j:
                    index = idx_i * num_channels + idx_j
                    pair_label = f"{i}_{j}_{freq_band}_plv"
                    data_dict[pair_label] = epoch[index]

        # convert the dictionary to df
        df = pd.DataFrame(data_dict)
        all_epochs.append(df)
    df_concat = pd.concat(all_epochs)
    return df_concat

In [18]:
channel_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
channel_labels = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15]

In [13]:
def channel_pairs(channels):
    pairs = []
    for i in range(len(channels)):
        for j in range(i + 1, len(channels)):
            pairs.append([channels[i], channels[j]])
    return pairs

In [14]:
channel_pair_idx = channel_pairs(channel_indices)

In [15]:
len(channel_pair_idx)

91